# Feature Creation
<font color='grey'>_Author @Oliver Zollikofer_</font><br>
__Kernel__: conda_python3


## Notebook outline

In this notebook we will build and push a docker image to ECR, create IAM roles and policies and run a simulation using ECS to simulate the requests to Keyspaces.
***

## Prerequisite

In order to run this Simulation, this notebook role requires some additional priviliges. There are related to ECR, IAM and ECS. Follow the steps to add these permissions to the notebook role:

1. Navigate to IAM in the console, and select __Policies__ in the left hand menu
2. Select __Create Policy__ and choose __JSON__ in the following menu. In the text area, paste the following code
```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowPushPull",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:BatchCheckLayerAvailability",
                "ecr:CompleteLayerUpload",
                "ecr:GetDownloadUrlForLayer",
                "ecr:InitiateLayerUpload",
                "ecr:PutImage",
                "ecr:UploadLayerPart"
            ],
            "Resource": [
                "arn:aws:ecr:*:*:repository/*"
            ]
        },
        {
            "Sid": "AllowIAMRoleCreation",
            "Effect": "Allow",
            "Action": [
                "iam:CreateRole",
                "iam:CreatePolicy",
                "iam:AttachRolePolicy",
                "iam:PassRole"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Sid": "AllowECSRoleCreation",
            "Effect": "Allow",
            "Action": [
                "ecs:CreateCluster",
                "ecs:CreateTaskSet",
                "ecs:ListClusters",
                "ecs:UpdateCluster",
                "ecs:DeleteCluster",
                "ecs:DeleteTaskSet",
                "ecs:RegisterTaskDefinition",
                "ecs:RunTask"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```
3. Select __Next__, you can fill in Tags as optional. Proceed to __Review__
4. Give the __Policy__ a name, you can call it _featureStoreSimulationRole_
5. Navigate to Sagemaker, and select the Notebook instance you are running. 
6. Under __Permissions and encryption__ select the IAM role the notebook is running with
7. Choose __Trust Relationships__
8. Click on __Edit trust relationship__ and paste the following json in the text field:
``` json
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": [
          "sagemaker.amazonaws.com",
          "ecs.amazonaws.com",
          "iam.amazonaws.com"
        ]
      },
      "Action": "sts:AssumeRole"
    }
  ]
}
```
9. Select __Permissions__ tab and select __Attach Policies__, search for _featureStoreSimulationRole_ (or the respective role if you named it differently)
10. Attach the policy and you should be able to run the simulation code
___


# 1. Configurations

### Import Libraries

In [1]:
import boto3
import json

### Set Clients

In [2]:
ecs_client = boto3.client('ecs')
iam_client = boto3.client('iam')
cw_client = boto3.client('logs')

### Set Variables
Please enter the subnet you want your ECS Task to run in, as well as Keyspaces username and password and S3 bucket name. For the other variables, you can change these according to your
needs, but it is advised to keep the variables in order to be able to reproduce the experiment

In [3]:
keyspaces_username = <put Keyspaces username here> #Keyspaces username (string)
keyspaces_password = <put Keyspaces username here> #Keyspaces password (string)
s3_bucket_name = <put S3 bucket name> #S3 bucket name (string)
subnet_ecs = <put the subnet to run the ecs task here, string format> # e.g. 'subnet-....'
project_name = 'simulation_keyspaces'
ecr_repository = 'simulation-keyspaces'
ecs_cluster_name = 'simulation_keyspaces'
ecs_cpu = '512'
ecs_memory = '1024'
worker_node_count = 1
ecs_task_role_name = 'ecs_simulation_keyspaces_task_role'
ecs_execution_role_name = 'ecs_simulation_keyspaces_execution_role'
cw_logGroupName='/ecs/simulation_keyspaces'

### Create Log group for ECS

In [ ]:
response_cw_log_group = cw_client.create_log_group(
    logGroupName=cw_logGroupName
)

### Set ECR Image Name

In [ ]:
# ECR Image URI
account_id = boto3.client('sts').get_caller_identity().get('Account')
tag = ':latest'
region = boto3.session.Session().region_name
uri_suffix = 'amazonaws.com'
if region in ['cn-north-1', 'cn-northwest-1']:
    uri_suffix = 'amazonaws.com.cn'
ecr_image_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

ecr_image_uri

# 2. Build and push docker image to ECR

_Requires permission to push image to ECR_

In [ ]:
%%sh
# Build and push docker image to ECR

# Specify an algorithm name
algorithm_name=simulation-keyspaces

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=YOUR_REGION-NAME-HERE

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

#$(aws ecr get-login --region ${region} --no-include-email)
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com
#docker login -u AWS ---p $(aws ecr get-login-password) ${account}.dkr.ecr.${region}.amazonaws.com
# Build the docker image locally with the image name and then push it to ECR
# with the full name.
cd container

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}
unset AWS_ACCESS_KEY_ID AWS_SECRET_ACCESS_KEY AWS_SESSION_TOKEN

# 3. Set up AWS Resources
Set up AWS Resources such as ECS Cluster, IAM Roles and Policies, ECS Task Definition

### Create ECS Cluster

_Requires permission to ECS Cluster_

In [4]:
#Create Cluster

response_cluster = ecs_client.create_cluster(
    clusterName='simulation_keyspaces',
    settings=[
        {
            'name': 'containerInsights',
            'value': 'enabled'
        },
    ]
)

### Create IAM Roles

#### Task Role

_Requires permission to create IAM Role_

In [ ]:
trust_relationship_policy_task = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": ["ecs.amazonaws.com",
                "ecs-tasks.amazonaws.com"]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

response_create_task_role = iam_client.create_role(
    RoleName=str(ecs_task_role_name),
    AssumeRolePolicyDocument=json.dumps(trust_relationship_policy_task)
)
response_create_task_role['Role']['Arn']

In [ ]:
policy_document_ecs_task = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": ["arn:aws:s3:::" + str(s3_bucket_name) + "/*"]
        },
        {
            "Action": [
                "autoscaling:Describe*",
                "cloudwatch:*",
                "logs:*",
                "iam:GetPolicy",
                "iam:GetPolicyVersion",
                "iam:GetRole"
            ],
            "Effect": "Allow",
            "Resource": "*"
        },
        {
            "Action": [
                "autoscaling:Describe*",
                "cloudwatch:*",
                "logs:*",
                "iam:GetPolicy",
                "iam:GetPolicyVersion",
                "iam:GetRole"
            ],
            "Effect": "Allow",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "iam:CreateServiceLinkedRole",
            "Resource": "arn:aws:iam::*:role/aws-service-role/events.amazonaws.com/AWSServiceRoleForCloudWatchEvents*",
            "Condition": {
                "StringLike": {
                    "iam:AWSServiceName": "events.amazonaws.com"
                }
            }
        }
    ]
    })

response_policy_task = iam_client.create_policy(
    PolicyName='ecs_simulation_task_policy',
    PolicyDocument = policy_document_ecs_task
)

response_policy_attach_task = iam_client.attach_role_policy(
    RoleName=str(ecs_task_role_name),
    PolicyArn=response_policy_task['Policy']['Arn']
)

#### Execution Role

_Requires permission to create IAM Role_

In [ ]:
trust_relationship_policy_execution = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "ecs-tasks.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

response_create_role_ecs_execution = iam_client.create_role(
    RoleName=str(ecs_execution_role_name),
    AssumeRolePolicyDocument=json.dumps(trust_relationship_policy_execution)
)


response_policy_attach_execution = iam_client.attach_role_policy(
    RoleName=str(ecs_execution_role_name),
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonECSTaskExecutionRolePolicy'
)

### Create ECS Task Definition

_Requires permission to create Task Definition_

In [9]:
# Create Task Definition
response_task_def = ecs_client.register_task_definition(
    family='simulation_keyspaces',
    taskRoleArn = response_create_task_role['Role']['Arn'],
    executionRoleArn =response_create_role_ecs_execution['Role']['Arn'],
    networkMode='awsvpc',
    containerDefinitions=[
        {
            "logConfiguration": {
                "logDriver": "awslogs",
                "options": {
                  "awslogs-group": cw_logGroupName,
                  "awslogs-region": str(region),
                  "awslogs-stream-prefix": "ecs"
                }
              },
            'name': str(project_name),
            'image': str(ecr_image_uri),
            "essential": True,
            "entryPoint": [],
            "command": []
        }    
    ],
    cpu = ecs_cpu,
    memory = ecs_memory
)

# 4. Run Simulation

_Add subnet to your VPC here_

In [ ]:
for worker_node in range(worker_node_count):
    
    response = ecs_client.run_task(

        cluster='simulation_keyspaces',
        count=1,
        launchType='FARGATE',
        networkConfiguration={
            'awsvpcConfiguration': {
                'subnets': [
                    str(subnet_ecs),
                ],
                'assignPublicIp': 'ENABLED'
            }
        },

        taskDefinition='simulation_keyspaces',




        overrides={
            'containerOverrides': [
                {
                    'name': 'simulation_keyspaces',
                    'environment': [
                        {
                            'name': 'workerNo',
                            'value': str(worker_node)
                        },
                        {
                            'name': 'requests',
                            'value': '10000', 
                        },
                        {
                            'name': 'num_processes',
                            'value': '1'
                        },
                        {
                            'name': 'simulation_name',
                            'value': 'Simulation_1_worker_10000_requests'
                        },
                        {
                            'name': 'keyspaces_username',
                            'value': str(keyspaces_username)
                        },
                        {
                            'name': 'keyspaces_password',
                            'value': str(keyspaces_password)
                        },
                        {
                            'name': 's3_bucket_name',
                            'value': str(s3_bucket_name)
                        }
                    ]
                }
            ]
        }
    )
    
    print(response)